<a href="https://colab.research.google.com/github/nisanuro/CNG483-Project1/blob/master/CNG483_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [341]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_images(path):
    images = []

    for filename in os.listdir(path):
        img=cv2.imread(os.path.join(image_path, filename),cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)

    return images

In [0]:
def image_to_feature_vector(image, size):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [0]:
def histogram(image, bin):
    bins = []
    width = int(256/bin)

    for low in range(0, bin * width, width):
        bins.append((low, low+width))

    hist = np.zeros(bin).astype('int32')
    image = image_to_feature_vector(image, (64, 64)).tolist()

    for value in image:
        for i in range(0, len(bins)):
            if (bins[i][0] <= value < bins[i][1]):
                hist[i] += 1
    
    return hist


    '''
    plt.figure(figsize=(12,4))
    index = np.arange(bin)
    plt.bar(index, hist)
    #plt.xlabel('Genre', fontsize=5)
    #plt.ylabel('No of Movies', fontsize=5)
    plt.xticks(index)
    plt.show()  '''   
    
    '''
    hist = img.flatten().tolist()
    grayscales  = np.unique(hist)
    frequencies = [hist.count(x) for x in grayscales]
    
    plt.figure(figsize=(12,4))
    #plt.bar(grayscales,frequencies,color='g')
    index = np.arange(bin)
    plt.bar(index, frequencies, color='g')
    #plt.hist(hist, density=True, bins=bin)
    #plt.show()

    '''
    '''
    hist = np.zeros(bin).astype('int32')
    binindices = (np.round((bin * img)/255)).astype('int64')
    for i in binindices:
        hist[i] += 1
    
    #plt.figure(figsize=(12,4))
    index = np.arange(bin)
    plt.bar(index, height = hist)
    #plt.xlabel('Genre', fontsize=5)
    #plt.ylabel('No of Movies', fontsize=5)
    plt.xticks(index)
    plt.show()'''

In [0]:
def color_histogram(img, bin):
    chans = cv2.split(img)
    colors = ("b", "g", "r")

    features = []
   
    for (chan, color) in zip(chans, colors):
        if color == 'b':
            bf = histogram(chan, bin)
        elif color == 'g':
            gf = histogram(chan, bin)
        else:
            rf = histogram(chan, bin)

    # combination of blue and red channel bins
    temp = np.transpose([np.tile(bf, len(gf)), np.repeat(gf, len(bf))])
    
    blue_green_hist = []    
    for i in temp:
        blue_green_hist.append(sum(i))

    # same operation with red channel bins
    temp = np.transpose([np.tile(rf, len(blue_green_hist)), np.repeat(blue_green_hist, len(rf))])

    color_hist = []    
    for i in temp:
        color_hist.append(sum(i))
    
    print(color_hist)

In [0]:
def split_image_into_grids(image, grid, i):
    
    image = cv2.resize(image, (256, 256))

    height=image.shape[0]
    width=image.shape[1]

    y1 = 0
    M = height//grid
    N = width//grid

    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M
            x1 = x + N
            grid = image[y:y+M, x:x+N]

            #cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0))

            #os.chdir("/content/drive/My Drive/SaveDeneme")
            #cv2.imwrite("im" + str(i) + '_' + str(x) + '_' + str(y)+".png",tiles)
            plt.figure()
            plt.imshow(grid) 
            plt.show()
    #cv2.imwrite("asas.png",image)        

In [0]:
def create_dataset(path):
    class_paths = []
    images = []
    labels = []

    with os.scandir(path) as itr: 
        for subdir in itr : 
            if subdir.is_dir():
                p = path + '/' + subdir.name
                class_paths.append(p)
    
    for p in class_paths:
        for filename in os.listdir(p):
            if filename.endswith(".jpg"):
                labels.append(os.path.basename(os.path.normpath(p)))                
                img = cv2.imread(os.path.join(p, filename))
                if img is not None:
                    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
   
    return images, labels


In [348]:
if __name__ == "__main__":

    test_path = "/content/drive/My Drive/CNG483-Project 1/TestSet"
    train_path = "/content/drive/My Drive/CNG483-Project 1/TrainingSet"
    validation_path = "/content/drive/My Drive/CNG483-Project 1/ValidationSet"

    
    #testI, testL = create_dataset(test_path)
    #trainI, trainL = create_dataset(train_path)
    validationI, validationL = create_dataset(validation_path)
    
    
    
    j = 1
    for i in validationI:
        gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
        #histogram(gray, 4)
        
        color_histogram(i, 2)
        #split_image_into_grids(i, 2, j)
        j +=1
        
        '''
        gray = gray.flatten().tolist()
        plt.figure(figsize=(12,4))
        plt.hist(gray, bins=4)
        plt.show()
        '''
      

[2314, 4854, 4886, 7426, 4862, 7402, 7434, 9974]
[10993, 7859, 7687, 4553, 7735, 4601, 4429, 1295]
[4526, 6904, 5594, 7972, 4316, 6694, 5384, 7762]
[4799, 5335, 6069, 6605, 5683, 6219, 6953, 7489]
[5419, 5943, 6015, 6539, 5749, 6273, 6345, 6869]
[5930, 5668, 6324, 6062, 6226, 5964, 6620, 6358]
[6464, 6252, 6252, 6040, 6248, 6036, 6036, 5824]
[1888, 4742, 4706, 7560, 4728, 7582, 7546, 10400]
[2457, 4993, 4883, 7419, 4869, 7405, 7295, 9831]
[8141, 6929, 6813, 5601, 6687, 5475, 5359, 4147]
[4727, 4681, 6353, 6307, 5981, 5935, 7607, 7561]
[4959, 5259, 6159, 6459, 5829, 6129, 7029, 7329]
[10459, 6575, 8211, 4327, 7961, 4077, 5713, 1829]
[7258, 3828, 8614, 5184, 7104, 3674, 8460, 5030]
[11359, 7353, 8327, 4321, 7967, 3961, 4935, 929]
[7070, 8932, 5286, 7148, 5140, 7002, 3356, 5218]
[10732, 6790, 7980, 4038, 8250, 4308, 5498, 1556]
[11239, 7281, 8127, 4169, 8119, 4161, 5007, 1049]
[7966, 7816, 4382, 4232, 8056, 7906, 4472, 4322]
[11237, 7279, 8129, 4171, 8117, 4159, 5009, 1051]
[1110, 3602, 5